In [32]:
# connect to the API
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
from env_vars import sentinel_username,sentinel_password

api = SentinelAPI(sentinel_username, sentinel_password, "https://scihub.copernicus.eu/apihub/")


# download single scene by known product id
# api.download("3f09e58e-b893-46fa-ad05-6bf1f5b4b257")

# search by polygon, time, and SciHub query keywords
footprint = geojson_to_wkt(read_geojson('./data/Geometry/congo_basin_boundary/congo_basin_boundary_custom_test.geojson'))
products = api.query(footprint,
                     date=('20150601', date(2015, 12, 31)),
                     raw='1C',
                     platformname='Sentinel-2',
                     cloudcoverpercentage=(0, 10))


# # # convert to Pandas DataFrame
products_df = api.to_dataframe(products)

# # GeoJSON FeatureCollection containing footprints and metadata of the scenes
# api.to_geojson(products)

# # GeoPandas GeoDataFrame with the metadata of the scenes and the footprints as geometries
# api.to_geodataframe(products)

# Get basic information about the product: its title, file size, MD5 sum, date, footprint and
# its download url
# api.get_product_odata("3f09e58e-b893-46fa-ad05-6bf1f5b4b257")

# # Get the product's full metadata available on the server
# api.get_product_odata(<product_id>, full=True)

Querying products: 100%|██████████| 246/246 [00:15<00:00, 15.83 products/s]


In [34]:
products_df

,title,link,link_alternative,link_icon,summary,datatakesensingstart,beginposition,endposition,ingestiondate,orbitnumber,...,platformname,size,tileid,hv_order_tileid,filename,identifier,uuid,level1cpdiidentifier,granuleidentifier,datastripidentifier
3fec66d7-7032-4efb-852d-0d8d674b1477,S2A_MSIL1C_20150708T081016_N0204_R078_T35NRC_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2015-07-08T08:10:16.027Z, Instrument: MS...",2015-07-08 08:10:16.027,2015-07-08 08:10:16.027,2015-07-08 08:10:16.027,2019-02-04 20:26:23.983,218,...,Sentinel-2,760.59 MB,35NRC,NC35R,S2A_MSIL1C_20150708T081016_N0204_R078_T35NRC_2...,S2A_MSIL1C_20150708T081016_N0204_R078_T35NRC_2...,3fec66d7-7032-4efb-852d-0d8d674b1477,S2A_OPER_MSI_L1C_TL_EPA__20160807T083950_A0002...,S2A_OPER_MSI_L1C_TL_EPA__20160807T083950_A0002...,S2A_OPER_MSI_L1C_DS_EPA__20160807T083950_S2015...
1aa532e4-7857-4ec6-bc53-d1a8432c2b9e,S2A_MSIL1C_20150708T081016_N0204_R078_T36NTH_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2015-07-08T08:10:16.027Z, Instrument: MS...",2015-07-08 08:10:16.027,2015-07-08 08:10:16.027,2015-07-08 08:10:16.027,2019-02-04 20:15:54.567,218,...,Sentinel-2,780.34 MB,36NTH,NH36T,S2A_MSIL1C_20150708T081016_N0204_R078_T36NTH_2...,S2A_MSIL1C_20150708T081016_N0204_R078_T36NTH_2...,1aa532e4-7857-4ec6-bc53-d1a8432c2b9e,S2A_OPER_MSI_L1C_TL_EPA__20160807T083950_A0002...,S2A_OPER_MSI_L1C_TL_EPA__20160807T083950_A0002...,S2A_OPER_MSI_L1C_DS_EPA__20160807T083950_S2015...
2519279e-4a14-40dc-9571-6996ac444e3d,S2A_MSIL1C_20150708T081016_N0204_R078_T35MQV_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2015-07-08T08:10:16.027Z, Instrument: MS...",2015-07-08 08:10:16.027,2015-07-08 08:10:16.027,2015-07-08 08:10:16.027,2019-02-04 20:14:41.739,218,...,Sentinel-2,766.07 MB,35MQV,MV35Q,S2A_MSIL1C_20150708T081016_N0204_R078_T35MQV_2...,S2A_MSIL1C_20150708T081016_N0204_R078_T35MQV_2...,2519279e-4a14-40dc-9571-6996ac444e3d,S2A_OPER_MSI_L1C_TL_EPA__20160807T083950_A0002...,S2A_OPER_MSI_L1C_TL_EPA__20160807T083950_A0002...,S2A_OPER_MSI_L1C_DS_EPA__20160807T083950_S2015...
3a58470b-5807-4a2a-8baa-6f172bda56c3,S2A_MSIL1C_20150708T081016_N0204_R078_T35NQD_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2015-07-08T08:10:16.027Z, Instrument: MS...",2015-07-08 08:10:16.027,2015-07-08 08:10:16.027,2015-07-08 08:10:16.027,2019-02-04 20:06:07.492,218,...,Sentinel-2,380.08 MB,35NQD,ND35Q,S2A_MSIL1C_20150708T081016_N0204_R078_T35NQD_2...,S2A_MSIL1C_20150708T081016_N0204_R078_T35NQD_2...,3a58470b-5807-4a2a-8baa-6f172bda56c3,S2A_OPER_MSI_L1C_TL_EPA__20160807T083950_A0002...,S2A_OPER_MSI_L1C_TL_EPA__20160807T083950_A0002...,S2A_OPER_MSI_L1C_DS_EPA__20160807T083950_S2015...
9f87d91b-5664-45a6-903b-cb0cd91d4571,S2A_MSIL1C_20150708T081016_N0204_R078_T36NTG_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2015-07-08T08:10:16.027Z, Instrument: MS...",2015-07-08 08:10:16.027,2015-07-08 08:10:16.027,2015-07-08 08:10:16.027,2019-02-04 20:05:07.275,218,...,Sentinel-2,755.37 MB,36NTG,NG36T,S2A_MSIL1C_20150708T081016_N0204_R078_T36NTG_2...,S2A_MSIL1C_20150708T081016_N0204_R078_T36NTG_2...,9f87d91b-5664-45a6-903b-cb0cd91d4571,S2A_OPER_MSI_L1C_TL_EPA__20160807T083950_A0002...,S2A_OPER_MSI_L1C_TL_EPA__20160807T083950_A0002...,S2A_OPER_MSI_L1C_DS_EPA__20160807T083950_S2015...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8068efd9-2169-4330-8736-3cfa4e8a7ebd,S2A_MSIL1C_20151230T090402_N0201_R007_T33MZQ_2...,https://sci

In [35]:
products_df.columns

Index(['title', 'link', 'link_alternative', 'link_icon', 'summary',
       'datatakesensingstart', 'beginposition', 'endposition', 'ingestiondate',
       'orbitnumber', 'relativeorbitnumber', 'cloudcoverpercentage',
       'gmlfootprint', 'format', 'instrumentshortname',
       'sensoroperationalmode', 'instrumentname', 'footprint', 's2datatakeid',
       'platformidentifier', 'orbitdirection', 'platformserialidentifier',
       'processingbaseline', 'processinglevel', 'producttype', 'platformname',
       'size', 'tileid', 'hv_order_tileid', 'filename', 'identifier', 'uuid',
       'level1cpdiidentifier', 'granuleidentifier', 'datastripidentifier'],
      dtype='object')

In [37]:
products_df_2 = products_df.drop_duplicates(subset='tileid')

In [38]:
len(products_df_2)

97

In [39]:
online = []
for product_id in products_df_2.index:
    if api.is_online(product_id):
        print('Product {} is online. Adding to list.'.format(product_id))
        online.append(product_id)
    else:
        continue

Product 3fec66d7-7032-4efb-852d-0d8d674b1477 is online. Adding to list.
Product 1aa532e4-7857-4ec6-bc53-d1a8432c2b9e is online. Adding to list.
Product 2519279e-4a14-40dc-9571-6996ac444e3d is online. Adding to list.
Product 3a58470b-5807-4a2a-8baa-6f172bda56c3 is online. Adding to list.
Product 9f87d91b-5664-45a6-903b-cb0cd91d4571 is online. Adding to list.
Product 41cce938-499c-441e-a897-3f7019a96674 is online. Adding to list.
Product 35e43b21-1b43-425b-b661-5425da5bb026 is online. Adding to list.
Product d37df6d6-7f15-4ab5-b24c-dbb6fe2b4a47 is online. Adding to list.
Product 52a4b387-7afa-49b9-80de-d8978ba49470 is online. Adding to list.
Product 207f0170-ef4c-44a3-b8ae-88a1f77d41b3 is online. Adding to list.
Product 327ed0f3-a44a-45aa-87dd-bc2ac0d9cce1 is online. Adding to list.
Product 6b71b44c-c0e1-4bdd-ba84-1e89b55e5450 is online. Adding to list.
Product 5d115e27-e0bb-4ba5-99d1-f341902e0be5 is online. Adding to list.
Product 2fe8f754-da6c-46e8-8983-be28b258cb17 is online. Adding t

In [40]:
len(online)

58

In [47]:
products_df_3 = products_df_2.loc[online]

In [48]:
len(products_df_3)

58

In [51]:
products_df_3["product_id"] = products_df_3.index

In [53]:
from collections import OrderedDict

# Get the unordered dictionary
unordered_products_dict = products_df_3.set_index('product_id').T.to_dict()

 # Then order it
ordered_dict = OrderedDict((k,unordered_products_dict.get(k)) for k in products_df_3.product_id)

In [55]:
ordered_dict['3fec66d7-7032-4efb-852d-0d8d674b1477']

{'title': 'S2A_MSIL1C_20150708T081016_N0204_R078_T35NRC_20150708T082158',
 'link': "https://scihub.copernicus.eu/apihub/odata/v1/Products('3fec66d7-7032-4efb-852d-0d8d674b1477')/$value",
 'link_alternative': "https://scihub.copernicus.eu/apihub/odata/v1/Products('3fec66d7-7032-4efb-852d-0d8d674b1477')/",
 'link_icon': "https://scihub.copernicus.eu/apihub/odata/v1/Products('3fec66d7-7032-4efb-852d-0d8d674b1477')/Products('Quicklook')/$value",
 'summary': 'Date: 2015-07-08T08:10:16.027Z, Instrument: MSI, Mode: , Satellite: Sentinel-2, Size: 760.59 MB',
 'datatakesensingstart': Timestamp('2015-07-08 08:10:16.027000'),
 'beginposition': Timestamp('2015-07-08 08:10:16.027000'),
 'endposition': Timestamp('2015-07-08 08:10:16.027000'),
 'ingestiondate': Timestamp('2019-02-04 20:26:23.983000'),
 'orbitnumber': 218,
 'relativeorbitnumber': 78,
 'cloudcoverpercentage': 3.0618,
 'gmlfootprint': '<gml:Polygon srsName="http://www.opengis.net/gml/srs/epsg.xml#4326" xmlns:gml="http://www.opengis.net/

In [56]:
products['3fec66d7-7032-4efb-852d-0d8d674b1477']

{'title': 'S2A_MSIL1C_20150708T081016_N0204_R078_T35NRC_20150708T082158',
 'link': "https://scihub.copernicus.eu/apihub/odata/v1/Products('3fec66d7-7032-4efb-852d-0d8d674b1477')/$value",
 'link_alternative': "https://scihub.copernicus.eu/apihub/odata/v1/Products('3fec66d7-7032-4efb-852d-0d8d674b1477')/",
 'link_icon': "https://scihub.copernicus.eu/apihub/odata/v1/Products('3fec66d7-7032-4efb-852d-0d8d674b1477')/Products('Quicklook')/$value",
 'summary': 'Date: 2015-07-08T08:10:16.027Z, Instrument: MSI, Mode: , Satellite: Sentinel-2, Size: 760.59 MB',
 'datatakesensingstart': datetime.datetime(2015, 7, 8, 8, 10, 16, 27000),
 'beginposition': datetime.datetime(2015, 7, 8, 8, 10, 16, 27000),
 'endposition': datetime.datetime(2015, 7, 8, 8, 10, 16, 27000),
 'ingestiondate': datetime.datetime(2019, 2, 4, 20, 26, 23, 983000),
 'orbitnumber': 218,
 'relativeorbitnumber': 78,
 'cloudcoverpercentage': 3.0618,
 'gmlfootprint': '<gml:Polygon srsName="http://www.opengis.net/gml/srs/epsg.xml#4326" 

In [58]:
# download all results from the search
api.download_all(ordered_dict,directory_path="./data/Sentinel_Data/2015/")

Downloading:   0%|          | 0.00/798M [00:00<?, ?B/s]
Downloading:   0%|          | 0.00/818M [00:00<?, ?B/s]
Downloading:   0%|          | 1.05M/818M [00:00<07:58, 1.71MB/s]
Downloading:   0%|          | 2.10M/818M [00:00<06:22, 2.13MB/s]
Downloading:   0%|          | 3.15M/818M [00:00<04:59, 2.72MB/s]
Downloading:   1%|          | 4.19M/818M [00:01<06:11, 2.19MB/s]
Downloading:   1%|          | 6.29M/818M [00:01<04:32, 2.98MB/s]
Downloading:   1%|          | 7.34M/818M [00:01<03:57, 3.42MB/s]
Downloading:   1%|          | 9.44M/818M [00:02<03:02, 4.44MB/s]
Downloading:   1%|▏         | 10.5M/818M [00:02<03:01, 4.45MB/s]
Downloading:   2%|▏         | 12.6M/818M [00:02<02:35, 5.19MB/s]
Downloading:   2%|▏         | 14.7M/818M [00:02<02:01, 6.59MB/s]
Downloading:   2%|▏         | 16.8M/818M [00:02<01:53, 7.09MB/s]
Downloading:   2%|▏         | 17.8M/818M [00:03<01:46, 7.51MB/s]
Downloading:   2%|▏         | 18.9M/818M [00:03<01:42, 7.84MB/s]
Downloading:   2%|▏         | 19.9M/818M [0

({'1aa532e4-7857-4ec6-bc53-d1a8432c2b9e': {'id': '1aa532e4-7857-4ec6-bc53-d1a8432c2b9e',
   'title': 'S2A_MSIL1C_20150708T081016_N0204_R078_T36NTH_20150708T082158',
   'size': 818289746,
   'md5': '1213E63E5D289D8B61DE2E2CCD69B22C',
   'date': datetime.datetime(2015, 7, 8, 8, 10, 16, 27000),
   'footprint': 'POLYGON((30.30179503793479 2.711148939858159,31.288890575770676 2.712957132034671,31.290030617657973 1.720022874436616,30.30359115375747 1.718876966821195,30.30179503793479 2.711148939858159))',
   'url': "https://scihub.copernicus.eu/apihub/odata/v1/Products('1aa532e4-7857-4ec6-bc53-d1a8432c2b9e')/$value",
   'Online': True,
   'Creation Date': datetime.datetime(2019, 2, 5, 13, 40, 12, 414000),
   'Ingestion Date': datetime.datetime(2019, 2, 4, 20, 15, 54, 567000),
   'path': './data/Sentinel_Data/2015/S2A_MSIL1C_20150708T081016_N0204_R078_T36NTH_20150708T082158.zip',
   'downloaded_bytes': 818289746},
  '3fec66d7-7032-4efb-852d-0d8d674b1477': {'id': '3fec66d7-7032-4efb-852d-0d8d6

In [57]:
help(api.download_all)

Help on method download_all in module sentinelsat.sentinel:

download_all(products, directory_path='.', max_attempts=10, checksum=True, n_concurrent_dl=2, lta_retry_delay=600) method of sentinelsat.sentinel.SentinelAPI instance
    Download a list of products.
    
    Takes a list of product IDs as input. This means that the return value of query() can be
    passed directly to this method.
    
    File names on the server are used for the downloaded files, e.g.
    "S1A_EW_GRDH_1SDH_20141003T003840_20141003T003920_002658_002F54_4DD1.zip".
    
    In case of interruptions or other exceptions, downloading will restart from where it left
    off. Downloading is attempted at most max_attempts times to avoid getting stuck with
    unrecoverable errors.
    
    Parameters
    ----------
    products : list
        List of product IDs
    directory_path : string
        Directory where the downloaded files will be downloaded
    max_attempts : int, optional
        Number of allowed retr